In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn import preprocessing, model_selection
import missingno as msno

In [ ]:
# Data importation
wvs_path = "gs://wvs-happiness-data/WVS_Time_Series_1981-2022_csv_v5_0.csv"
df = pd.read_csv(wvs_path,storage_options=)
df.head()

_request non-retriable exception: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist)., 401
Traceback (most recent call last):
  File "c:\Users\PC\anaconda3\Lib\site-packages\gcsfs\retry.py", line 132, in retry_request
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\PC\anaconda3\Lib\site-packages\gcsfs\core.py", line 470, in _request
    validate_response(status, contents, path, args)
  File "c:\Users\PC\anaconda3\Lib\site-packages\gcsfs\retry.py", line 119, in validate_response
    raise HttpError(error)
gcsfs.retry.HttpError: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist)., 401


HttpError: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist)., 401

##

# Data Preprocessing

In [3]:
# representing missing data by nan istead of numerical values
missing_values = [-1, -2,-3, -4, -5]
df.replace(missing_values, np.nan, inplace=True)

In [4]:
# Drop meta data and composed columns
df=df.drop(columns = [col for col in df.columns if col.startswith('Y')] )

# Data Cleaning
> Preparing data is a continuous process rather than a one-time task. 

### Handling Missing Data

In [5]:
# visualize Missing DATA 
missing_percentage = (df.isnull().sum() / len(df)) * 100
missing_percentage = missing_percentage[missing_percentage > 0]  
missing_percentage = missing_percentage.sort_values(ascending=False)  

print(missing_percentage)

F186       99.916345
A124_31    99.905973
G007_13    99.894924
F187       99.891542
E216       99.845994
             ...    
A042        0.710729
A038        0.692240
A029        0.674877
A035        0.584683
A032        0.563939
Length: 990, dtype: float64


what is the pattern of missing data ?
how to handle missing data?
what columns/rows to drop , what columns/rows to keep ? 
how to impute the missing values ? 
 Why are these values missing?
Is missingness random or systematic?
Does missingness affect the target variable?
Will imputing missing data introduce bias?
Can missingness itself be a feature?
What is the best imputation method based on data distribution?
Do I need domain knowledge to understand why data is missing?

In [6]:
# Dataset after missing data handling 


# Feature Selection 

# predicting happiness 

Predicting the features :
- A008 (Feeling of happiness) 
- A170 (Satisfaction with life) 


In [7]:
for col in ['A008','A170']:
   print(df[col].unique())

[ 3.  2.  1.  4. nan]
[ 3.  9.  4.  7.  5.  2.  8.  6.  1. nan 10.]


## Logistic Regression 


In [8]:
# Ordinal Logistic Regression  (Proportional Odds Model)

In [9]:
# Random Forest / Decision Tree

In [10]:
# XGboost 

In [11]:
# CatGboost 

In [12]:
# LightGboost

In [13]:
# Ordinal SVM 

In [14]:
# Neural Networks for Ordinal Data ( DeepLearning)

# Detailed analysis on income

# Statistical tests 

# Happiness Trends Over Time (1981-2022)

# Sentiment Analysis on Open-Ended Responses (NLP)